## 仅供交叉验证 Encoder-Decoder with LSTM cell-按总量分类

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from utils import *
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
n_input = 11

读取数据

In [3]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [4]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


### 规范数据并获取5折交叉检验所需的训练集和验证集

In [46]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 按总量切分数据

In [2]:
def split_data_by_es(data, targets):
    total_es = np.sum(data[:, :11, -2], axis=1)
    sorted_index = np.argsort(total_es)
    group_size = len(total_es) // 3
    
    data1, target1 = data[sorted_index[:group_size]], targets[sorted_index[:group_size]]
    data2, target2 = data[sorted_index[group_size:2*group_size]], targets[sorted_index[group_size:2*group_size]]
    data3, target3 = data[sorted_index[2*group_size:]], targets[sorted_index[2*group_size:]]
    
    return data1, target1, data2, target2, data3, target3

### 构建模型

In [3]:
def root_mean_squared_error(y_true, y_pred):
        return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true), axis=-1)) 

def build_encoder_decoder_model(lstm_units, dense_units, lr=1e-4):
    model = keras.models.Sequential()
    model.add(LSTM(lstm_units, activation='tanh', input_shape=(11, 10), return_sequences=True))
    model.add(LSTM(lstm_units, activation='tanh'))
    model.add(RepeatVector(5))
    model.add(LSTM(lstm_units, activation='tanh', return_sequences=True))
    model.add(LSTM(lstm_units, activation='tanh', return_sequences=True))
    model.add(TimeDistributed(Dense(dense_units, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    
    optimizer=keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss=root_mean_squared_error, optimizer=optimizer)
    return model

### 进行训练和评估
使用EarlyStopping和Checkpoint做训练停止方式

In [4]:
def cross_validation(X, y, y_cat, kfold, scaler):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }
    
    for train, test in kfold.split(X, y_cat):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        models = []
        
        # 按总量划分数据集
        X_train1, y_train1, X_train2, y_train2, X_train3, y_train3 = split_data_by_es(X_train, y_train)
        train_xs = [X_train1, X_train2, X_train3]
        train_ys = [y_train1, y_train2, y_train3]
        
        X_test1, y_test1, X_test2, y_test2, X_test3, y_test3 = split_data_by_es(X_test, y_test)
        test_xs = [X_test1, X_test2, X_test3]
        test_ys = [y_test1, y_test2, y_test3]
        i_s = [1, 2, 3]
        
        # 训练
        for i in range(len(i_s)):
            model = build_encoder_decoder_model(256, 256, 1e-4)
            history = model.fit(train_xs[i], train_ys[i], epochs=100, batch_size=16, verbose=1, validation_data=(test_xs[i], test_ys[i]),
                           callbacks=[
                               EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto', restore_best_weights=True)
                           ])
            models.append(model)
        
        # 预测
        y_test = []
        y_pred = []
        for i in range(len(i_s)):
            y_test.append(test_ys[i])
            y_pred.append(models[i].predict(test_xs[i]).reshape(test_ys[i].shape))
        
        y_test = np.concatenate(y_test)
        y_pred = np.concatenate(y_pred)

        metrics = ['mae', 'rmse','ndcg', 'mape', 'r2', 'pearson', 'acc']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics

In [5]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler)
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
    pickle.dump(metrics, open('rnn_metrics.dict', 'wb'))
    
    return metrics

In [6]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)
Train on 704 samples, validate on 177 samples
Epoch 1/100
704/704 [==============================] - 5s 6ms/sample - loss: 0.8927 - val_loss: 0.7192
Epoch 2/100
704/704 [==============================] - 0s 334us/sample - loss: 0.8190 - val_loss: 0.7021
Epoch 3/100
704/704 [==============================] - 0s 340us/sample - loss: 0.8134 - val_loss: 0.6924
Epoch 4/100
704/704 [==============================] - 0s 405us/sample - loss: 0.7972 - val_loss: 0.6788
Epoch 5/100
704/704 [==============================] - 1s 736us/sample - loss: 0.7934 - val_loss: 0.6778
Epoch 6/100
704/704 [==============================] - 1s 754us/sample - loss: 0.7895 - val_loss: 0.6724
Epoch 7/100
704/704 [==============================] - 1s 1ms/sample - loss: 0.7879 - val_loss: 0.6692
Epoch 8/100
704/704 [==============================] - 1s 1ms/sample - loss: 0.7849 - val_loss: 0.6696
Epoch 9/100
704/704 [=====

704/704 [==============================] - 1s 2ms/sample - loss: 0.7944 - val_loss: 0.8284
Epoch 50/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7940 - val_loss: 0.8285
Epoch 51/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7939 - val_loss: 0.8275
Epoch 52/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7943 - val_loss: 0.8287
Epoch 53/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7946 - val_loss: 0.8284
Epoch 54/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7978 - val_loss: 0.8309
Epoch 55/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7948 - val_loss: 0.8300
Epoch 56/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7938 - val_loss: 0.8338
Epoch 57/100
704/704 [==============================] - 1s 1ms/sample - loss: 0.7950 - val_loss: 0.8305
Epoch 58/100
704/704 [==============================] - 1s 1ms/sample - loss:

Epoch 19/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7561 - val_loss: 0.7565
Epoch 20/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7531 - val_loss: 0.7538
Epoch 21/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7517 - val_loss: 0.7548
Epoch 22/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7517 - val_loss: 0.7495
Epoch 23/100
704/704 [==============================] - 1s 2ms/sample - loss: 0.7528 - val_loss: 0.7481
Epoch 24/100
704/704 [==============================] - 1s 847us/sample - loss: 0.7514 - val_loss: 0.7499
Epoch 25/100
704/704 [==============================] - 1s 1ms/sample - loss: 0.7521 - val_loss: 0.7474
Epoch 26/100
704/704 [==============================] - 1s 1ms/sample - loss: 0.7556 - val_loss: 0.7518
Epoch 27/100
704/704 [==============================] - 1s 1ms/sample - loss: 0.7534 - val_loss: 0.7506
Epoch 28/100
704/704 [==============================] - 1s 1ms

705/705 [==============================] - 1s 1ms/sample - loss: 1.1102 - val_loss: 1.1709
Epoch 24/100
705/705 [==============================] - 1s 2ms/sample - loss: 1.1097 - val_loss: 1.1699
Epoch 25/100
705/705 [==============================] - 1s 2ms/sample - loss: 1.1101 - val_loss: 1.1711
Epoch 26/100
705/705 [==============================] - 1s 2ms/sample - loss: 1.1066 - val_loss: 1.1708
Epoch 27/100
705/705 [==============================] - 1s 2ms/sample - loss: 1.1062 - val_loss: 1.1692
Epoch 28/100
705/705 [==============================] - 1s 2ms/sample - loss: 1.1055 - val_loss: 1.1709
Epoch 29/100
705/705 [==============================] - 1s 2ms/sample - loss: 1.1068 - val_loss: 1.1649
Epoch 30/100
705/705 [==============================] - 1s 2ms/sample - loss: 1.1087 - val_loss: 1.1677
Epoch 31/100
705/705 [==============================] - 1s 2ms/sample - loss: 1.1056 - val_loss: 1.1633
Epoch 32/100
705/705 [==============================] - 1s 2ms/sample - loss:

705/705 [==============================] - 1s 1ms/sample - loss: 0.7887 - val_loss: 0.8260
Epoch 26/100
705/705 [==============================] - 1s 1ms/sample - loss: 0.7897 - val_loss: 0.8190
Epoch 27/100
705/705 [==============================] - 1s 1ms/sample - loss: 0.7882 - val_loss: 0.8208
Epoch 28/100
705/705 [==============================] - 1s 2ms/sample - loss: 0.7915 - val_loss: 0.8178
Epoch 29/100
705/705 [==============================] - 1s 2ms/sample - loss: 0.7890 - val_loss: 0.8166
Epoch 30/100
705/705 [==============================] - 1s 936us/sample - loss: 0.7882 - val_loss: 0.8159
Epoch 31/100
705/705 [==============================] - 1s 2ms/sample - loss: 0.7917 - val_loss: 0.8158
Epoch 32/100
705/705 [==============================] - 1s 2ms/sample - loss: 0.7873 - val_loss: 0.8209
Epoch 33/100
705/705 [==============================] - 1s 2ms/sample - loss: 0.7862 - val_loss: 0.8150
Epoch 34/100
705/705 [==============================] - 1s 2ms/sample - los

...

Epoch 36/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4179 - val_loss: 0.4542
Epoch 37/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4164 - val_loss: 0.4546
Epoch 38/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4173 - val_loss: 0.4531
Epoch 39/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4168 - val_loss: 0.4529
Epoch 40/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4159 - val_loss: 0.4522
Epoch 41/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4159 - val_loss: 0.4523
Epoch 42/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4162 - val_loss: 0.4530
Epoch 43/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4157 - val_loss: 0.4525
Epoch 44/100
1370/1370 [==============================] - 2s 2ms/sample - loss: 0.4148 - val_loss: 0.4517
Epoch 45/100
1370/1370 [======================

1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3972 - val_loss: 0.3667
Epoch 17/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3975 - val_loss: 0.3646
Epoch 18/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3979 - val_loss: 0.3640
Epoch 19/100
1371/1371 [==============================] - 1s 438us/sample - loss: 0.3959 - val_loss: 0.3638
Epoch 20/100
1371/1371 [==============================] - 1s 564us/sample - loss: 0.3955 - val_loss: 0.3644
Epoch 21/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3966 - val_loss: 0.3648
Epoch 22/100
1371/1371 [==============================] - 2s 1ms/sample - loss: 0.3942 - val_loss: 0.3644
Epoch 23/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3960 - val_loss: 0.3635
Epoch 24/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3956 - val_loss: 0.3666
Epoch 25/100
1371/1371 [==============================]

1372/1372 [==============================] - 2s 2ms/sample - loss: 0.9530 - val_loss: 0.9420
Epoch 22/100
1372/1372 [==============================] - 2s 2ms/sample - loss: 0.9514 - val_loss: 0.9454
Epoch 23/100
1372/1372 [==============================] - 2s 2ms/sample - loss: 0.9545 - val_loss: 0.9426
Epoch 24/100
1372/1372 [==============================] - 2s 2ms/sample - loss: 0.9526 - val_loss: 0.9405
Epoch 25/100
1372/1372 [==============================] - 2s 2ms/sample - loss: 0.9544 - val_loss: 0.9449
Epoch 26/100
1372/1372 [==============================] - 2s 1ms/sample - loss: 0.9525 - val_loss: 0.9466
Epoch 27/100
1372/1372 [==============================] - 2s 2ms/sample - loss: 0.9512 - val_loss: 0.9417
Epoch 28/100
1372/1372 [==============================] - 2s 2ms/sample - loss: 0.9515 - val_loss: 0.9410
Epoch 29/100
1372/1372 [==============================] - 2s 2ms/sample - loss: 0.9506 - val_loss: 0.9413
Epoch 30/100
1372/1372 [==============================] - 2

Epoch 45/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3813 - val_loss: 0.4147
Epoch 46/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3787 - val_loss: 0.4166
Epoch 47/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3791 - val_loss: 0.4167
Epoch 48/100
1371/1371 [==============================] - 2s 1ms/sample - loss: 0.3796 - val_loss: 0.4160
Epoch 49/100
1371/1371 [==============================] - 2s 1ms/sample - loss: 0.3791 - val_loss: 0.4141
Epoch 50/100
1371/1371 [==============================] - 2s 1ms/sample - loss: 0.3791 - val_loss: 0.4169
Epoch 51/100
1371/1371 [==============================] - 2s 1ms/sample - loss: 0.3794 - val_loss: 0.4154
Epoch 52/100
1371/1371 [==============================] - 2s 1ms/sample - loss: 0.3802 - val_loss: 0.4172
Epoch 53/100
1371/1371 [==============================] - 2s 2ms/sample - loss: 0.3807 - val_loss: 0.4172
Epoch 54/100
1371/1371 [======================

1373/1373 [==============================] - 2s 2ms/sample - loss: 0.9540 - val_loss: 0.9718
Epoch 12/100
1373/1373 [==============================] - 2s 2ms/sample - loss: 0.9527 - val_loss: 0.9701
Epoch 13/100
1373/1373 [==============================] - 2s 2ms/sample - loss: 0.9525 - val_loss: 0.9696
Epoch 14/100
1373/1373 [==============================] - 1s 992us/sample - loss: 0.9539 - val_loss: 0.9708
Epoch 15/100
1373/1373 [==============================] - 1s 906us/sample - loss: 0.9506 - val_loss: 0.9676
Epoch 16/100
1373/1373 [==============================] - 1s 910us/sample - loss: 0.9511 - val_loss: 0.9774
Epoch 17/100
1373/1373 [==============================] - 2s 1ms/sample - loss: 0.9494 - val_loss: 0.9775
Epoch 18/100
1373/1373 [==============================] - 2s 1ms/sample - loss: 0.9516 - val_loss: 0.9672
Epoch 19/100
1373/1373 [==============================] - 2s 1ms/sample - loss: 0.9493 - val_loss: 0.9671
Epoch 20/100
1373/1373 [=============================

In [7]:
metrics

{'gene': {'overall': {'mae': 0.730133398463071,
   'rmse': 1.2883865069292366,
   'ndcg': 0.4595261782617205,
   'mape': 4.236438889071694,
   'r2': 0.24111341230011735,
   'pearson': 0.5244351917116782,
   'acc': 0.3585302695937836},
  'annual': {'mae': array([0.4229301 , 0.57894903, 0.7480632 , 0.88135736, 1.0193673 ]),
   'rmse': array([0.77587887, 0.97472237, 1.31649913, 1.47399919, 1.66308958]),
   'ndcg': array([0.51792661, 0.46710419, 0.16252051, 0.27382767, 0.1715413 ]),
   'mape': array([3.00807798, 2.71817289, 3.09897554, 5.97081394, 6.3861541 ]),
   'r2': array([0.39648012, 0.27090543, 0.18840093, 0.06966091, 0.02438854]),
   'pearson': array([0.65189546, 0.55748179, 0.47756879, 0.33801751, 0.28149024]),
   'acc': array([0.61641129, 0.2852523 , 0.24175685, 0.33479559, 0.31443532])}},
 'transplant': {'overall': {'mae': 0.758002721263924,
   'rmse': 1.2550889729706527,
   'ndcg': 0.5085506950352213,
   'mape': 3.6212566596977545,
   'r2': 0.4375413865835555,
   'pearson': 0.66

In [49]:
overall_metrics

{'mae': [0.7593994116060747,
  0.7671028942138409,
  0.7561637531690426,
  0.7418184505992778,
  0.754409714862091],
 'rmse': [1.2462465377385148,
  1.2449577006458816,
  1.2305761200091225,
  1.2152881768708477,
  1.3016089605016066],
 'ndcg': [0.12383840192943663,
  0.8889930008127779,
  0.5801052823760275,
  0.7835049919448109,
  0.07865601010951555]}